In [ ]:
# Radar chart URL - https://python-graph-gallery.com/390-basic-radar-chart/
# list of references and ideas - https://towardsdatascience.com/jupyter-superpower-interactive-visualization-combo-with-python-ffc0adb37b7b
# https://observablehq.com/@rreusser/domain-coloring-for-complex-functions?collection=@observablehq/webgl
# http://users.mai.liu.se/hanlu09/complex/domain_coloring.html
# more radar https://python-graph-gallery.com/390-basic-radar-chart/
# more radar https://matplotlib.org/examples/api/radar_chart.html

In [ ]:
#%pip install geoip2

#https://dev.maxmind.com/geoip/geoip2/geolite2/
#This product includes GeoLite2 data created by MaxMind, available from
#<a href="http://www.maxmind.com">http://www.maxmind.com</a>

import geoip2.database
reader = geoip2.database.Reader('Tools/GeoLite2-City.mmdb')

#create a function to find the lat/long of the addy based on maxmind reader
def lookup_location(IPAddress):
    try:
        response = reader.city(IPAddress)
        return round(response.location.latitude,3), round(response.location.longitude,3), response.country.name, response.city.name
    except:
        # If private IP address, return address at center of map
        return 0,0,0,0
        #return null

#function to get a dual output into different columns in a dataframe
#https://stackoverflow.com/questions/23690284/pandas-apply-function-that-returns-multiple-values-to-rows-in-pandas-dataframe
def apply_and_concat(dataframe, field, func, column_names):
    return pd.concat((
                         dataframe,
                         dataframe[field].apply(
                         lambda cell: pd.Series(func(cell), index=column_names))), axis=1)

def get_location(df_ip,columnIP):
    df_min = apply_and_concat(df_ip, columnIP, lookup_location, ['latitude', 'longitude', 'country', 'state'])
    # Remove the rows with missing lat/long
    df_min = df_min[(df_min.latitude != 0.000)&(df_min.longitude != 0.000)]
    return df_min

In [ ]:
# pip install gmaps
#https://jupyter-gmaps.readthedocs.io/en/latest/tutorial.html
import gmaps
import gmaps.datasets

# Enter your Google Maps API key. You could use pickle to serialize it and store it outside of code.
api = 'EnterYourGoogleMapsAPI'
gmaps.configure(api_key=api)

def prepare_location(df_min):
    #pandas chaining
    #1 - group by lat/long
    #2 - get the size of a column to consolidate groupby object
    #3 - take it from a indexed series to a dataframe
    #4 - reset the index so that groupby is flattened and can be referenced
    #5 - rename the size column (0) to 'count'
    df_plot = df_min.groupby(['latitude', 'longitude']).size().to_frame().reset_index().rename(columns={0:'count'})
    return df_plot

# Need to enable extensions and widgets before figure will display
#jupyter nbextension enable --py gmaps
#jupyter nbextension enable --py widgetsnbextension
#Restart Jupyter

def get_heatmap(df_plot):
    locations = df_plot[['latitude', 'longitude']]
    weights = df_plot['count']
    fig = gmaps.figure()
    #fig.add_layer(gmaps.heatmap_layer(locations, weights=weights))
    fig.add_layer(gmaps.heatmap_layer(locations))
    return fig